<a href="https://colab.research.google.com/github/kojurkin/NonparametricsStat/blob/main/NonparametricsStat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.special import gamma, gammaln
from scipy.integrate import quad

In [ ]:
def empirical_cdf(x, sample):
    n = len(sample)
    return np.sum(sample <= x) / n

In [ ]:
def bessel_mod(v, z, k_max=100, tolerance=1e-6):
    total_sum = 0.0
    z_half = z / 2.0
    for k in range(k_max):
        log_term = (v + 2 * k) * np.log(abs(z / 2.0)) - gammaln(k + 1) - gammaln(k + v + 1)
        term = np.exp(log_term)
        total_sum += term
        if abs(term) < tolerance:
            break
    return total_sum

In [ ]:
def integrand(y, t, j):
  return np.exp((t/(8*(y**2 + 1))) - (((4*j+1)**2 * np.pi**2 * y**2)/(8*t)))

In [ ]:
def compute_ranks(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    # (элемент, выборка, исходный_индекс)
    combined = [(x, 0, i) for i, x in enumerate(sample1)] + [(x, 1, i) for i, x in enumerate(sample2)]
    sorted_data = sorted(combined, key=lambda x: x[0])
    n = len(combined)
    ranks = [0] * n
    i = 0
    while i < n:
        start = i
        current_value = sorted_data[i][0]
        count = 0
        while i < n and sorted_data[i][0] == current_value:
            i += 1
            count += 1
        avg_rank = (start + 1 + start + count) / 2
        for j in range(start, i):
            ranks[sorted_data[j][2] + (sorted_data[j][1] * len(sample1))] = avg_rank
    ranks1 = []
    ranks2 = []
    for item in sorted(combined, key=lambda x: x[2]):
        rank_index = item[2] + (item[1] * len(sample1))
        if item[1] == 0:
            ranks1.append(ranks[rank_index])
        else:
            ranks2.append(ranks[rank_index])
    return np.array(ranks1), np.array(ranks2)

In [ ]:
def a1function(t, j_max=1000, tolerance=1e-16, round_to_5 = True):
    if t <= 0:
        return 0.0
    total_sum = 0.0
    sqrt_2t = np.sqrt(2 * t)

    for j in range(j_max):
        term1 = gamma(j + 0.5) * np.sqrt(4 * j + 1)
        term2 = gamma(0.5) * gamma(j + 1)
        exponent = np.exp(-(4 * j + 1)**2 / (16 * t))
        arg = (4 * j + 1)**2 / (16 * t)
        bessel_diff = bessel_mod(-0.25, arg, k_max=173, tolerance=tolerance) - bessel_mod(0.25, arg, k_max=173, tolerance=tolerance)

        term = (term1 / term2) * exponent * bessel_diff

        total_sum += term

        if abs(term) < tolerance:
            break
    result = total_sum / np.sqrt(2 * t)

    if round_to_5:
        result = round(result, 5)
    return result

In [ ]:
def a2function(t, j_max=1000, tolerance=1e-16, round_to_5 = True):
    if t <= 0:
        return 0.0
    total_sum = 0.0

    for j in range(j_max):
        term1 = gamma(j + 0.5) * (4 * j + 1)
        term2 = gamma(0.5) * gamma(j + 1)
        exponent = np.exp(-1 * ((4 * j + 1)**2 * np.pi**2) / (8 * t))
        integral_exponent, tolerance_message = quad(integrand, 0, np.inf, args=(t, j))

        term = (-1)**j * (term1 / term2) * exponent * integral_exponent

        total_sum += term

        if abs(term) < tolerance:
            break

    result = total_sum * (np.sqrt(2 * np.pi) / t)
    if round_to_5:
        result = round(result, 5)
#    print(tolerance_message)
    return result

In [ ]:
def w_squared_test(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    n = len(sample1)
    sample2_sorted = np.sort(sample2)
    sample2_cdf = np.vectorize(lambda x: empirical_cdf(x, sample2))
    U = sample2_cdf(sample1)
    U_sorted = np.sort(U)
    sum_term = 0
    for i in range(n):
        diff = U_sorted[i] - (2 * (i + 1) - 1) / (2 * n)
        sum_term += diff ** 2
    w_squared = ((1 / (12 * n)) + sum_term)
    p_value = 1 - a1function(w_squared)
    return w_squared, p_value

In [ ]:
def lehmann_rosenblatt_test(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    m, n = len(sample1), len(sample2)
    if m > n:
        tmp = sample2
        sample2 = sample1
        sample1 = tmp
        m, n = len(sample1), len(sample2)
    ranks1, ranks2 = compute_ranks(sample1, sample2)
    sum1 = 0
    sum2 = 0
    ranks1 = sorted(ranks1)
    ranks2 = sorted(ranks2)
    for i in range(len(ranks1)):
      sum1 += (ranks1[i] - (i + 1))**2
    for i in range(len(ranks2)):
      sum2 += (ranks2[i] - (i + 1))**2
    T = (n * sum1 + m * sum2) / (m * n * (m + n)) - ((4 * m * n - 1) / (6 * (m + n)))
    p_value = 1 - a1function(T)
    return T, p_value

In [ ]:
def anderson_darling_test(sample1, sample2):
    if len(sample1) == 0 or len(sample2) == 0:
        raise ValueError("Выборки должны быть не пустыми.")
    m, n = len(sample1), len(sample2)
    if m > n:
      tmp = sample2
      sample2 = sample1
      sample1 = tmp

    combined = np.concatenate([sample1, sample2])
    sorted_combined = np.sort(combined)

    ranks1, ranks2 = compute_ranks(sample1, sample2)

    M = []
    for j in range(m + n):
        value = sorted_combined[j]
        M_j = sum(1 for x in sample1 if x <= value)
        M.append(M_j)
    total = 0.0
    for i in range(1, m + n):
        term = ((M[i-1] * (m + n) - m*i) ** 2) / (i * (m + n - i))
        total += term

    A = total / (m * n)
    p_value = 1 - a2function(A)
    return A, p_value

In [ ]:
def kolmogorov_smirnov_test(data1, data2):
    data1 = np.sort(data1)
    data2 = np.sort(data2)
    n1 = len(data1)
    n2 = len(data2)
    data_all = np.concatenate([data1, data2])
    cdf1 = np.searchsorted(data1, data_all, side='right') / np.float64(n1)
    cdf2 = np.searchsorted(data2, data_all, side='right') / np.float64(n2)
    d = np.float64(np.max(np.abs(cdf1 - cdf2)))
    return d, np.clip(kolmogorov_distribution(d, n1, n2), 0.0, 1.0),

In [ ]:
def kolmogorov_distribution(d, n1, n2):
    d = float(d)
    n1 = int(n1)
    n2 = int(n2)

    if n1 <= 10 and n2 <= 10:
        m, n = n1, n2
        k = int(np.ceil(d * n * m))
        if k == 0:
            return 1.0
        dp = np.zeros((n + 1, m + 1, k + 1), dtype=np.float64)
        dp[0, 0, 0] = 1.0
        for i in range(n + 1):
            for j in range(m + 1):
                for t in range(k + 1):
                    if dp[i, j, t] == 0:
                        continue
                    diff = abs(i / np.float64(n) - j / np.float64(m))
                    if diff * n * m > t:
                        continue
                    if i < n:
                        dp[i + 1, j, t] += dp[i, j, t] / np.float64(n)
                    if j < m:
                        dp[i, j + 1, t] += dp[i, j, t] / np.float64(m)
        p = 0.0
        for t in range(k):
            p += dp[n, m, t]
        return np.clip(1 - p, 0.0, 1.0)
    else:
        m, n = sorted([float(n1), float(n2)], reverse=True)
        en = m * n / (m + n)
        z = d * np.sqrt(en)
        z = np.asarray(z, dtype=np.float64)
        p = np.zeros_like(z)
        mask_non_positive = z <= 0
        p[mask_non_positive] = 1.0
        mask_inf = np.isinf(z)
        p[mask_inf] = 0.0
        mask_valid = (~mask_non_positive) & (~mask_inf)
        z_valid = z[mask_valid]
        if z_valid.size > 0:
            mask_small = z_valid < 1.18
            if np.any(mask_small):
                sqrt_2pi = np.sqrt(2 * np.pi)
                z_small = z_valid[mask_small]
                p_small_sum = np.zeros_like(z_small)
                for k in range(1, 1000):
                    term = -(2*k - 1)**2 * np.pi**2 / (8 * z_small**2)
                    if np.any(term < -708.3964185322641):
                        break
                    current = np.where(term == -np.inf, 0.0, np.exp(np.clip(term, -708.3964185322641, 709.782712893384)))
                    p_small_sum += current
                    if np.all(np.abs(current) < 1e-30):
                        break
                raw_p_small = 1.0 - sqrt_2pi / z_small * p_small_sum
                p_small = np.clip(raw_p_small, 0.0, 1.0)
                if z.ndim == 0:
                    if mask_small:
                        p = p_small
                else:
                    p[mask_valid & mask_small] = p_small
            mask_large = ~mask_small
            if np.any(mask_large):
                z_large = z_valid[mask_large]
                p_large_sum = np.zeros_like(z_large)
                for k in range(1, 1000):
                    term = -2 * k**2 * z_large**2
                    if np.all(term < -708.3964185322641):
                        break
                    sign = 1.0 if k % 2 == 1 else -1.0
                    current = sign * np.where(term == -np.inf, 0.0, np.exp(np.clip(term, -708.3964185322641, 709.782712893384)))
                    p_large_sum += current
                    if np.all(np.abs(current) < 1e-30):
                        break
                raw_p_large = 2.0 * p_large_sum
                p_large = np.clip(raw_p_large, 0.0, 1.0)
                if z.ndim == 0:
                    if mask_large:
                        p = p_large
                else:
                    p[mask_valid & mask_large] = p_large
        return p.item() if z.ndim == 0 else p